# Criptoaritmética

In [ ]:
from copy import deepcopy
from utiles import *

## Definición de formal como clase

In [ ]:
class Criptoaritmetica:
    """
    Problema de criptoaritmetica, el cual consiste en encontrar
    el valor de todas las letras que aparecen en una operación de suma criptoaritmetica
    """
    def __init__(self,sumando1:str,sumando2:str,resultado:str):
        alfabeto = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

        # inicializacion
        self.solucion = dict()
        self.sumandos = [sumando1.upper(), sumando2.upper()]
        self.resultado = resultado.upper()
        
        for c in self.sumandos[0]:
            assert c in alfabeto, "ERROR: Algun caracter del primer sumando no esta en el alfabeto"
        for c in self.sumandos[1]:
            assert c in alfabeto, "ERROR: Algun caracter del segundo sumando no esta en el alfabeto"
        for c in self.resultado:
            assert c in alfabeto, "ERROR: Algun caracter del resultado no esta en el alfabeto"
        
        print(f"Operacion final: {self.sumandos[0]}+{self.sumandos[1]}={self.resultado}")
        
        # Representacion del estado inicial por medio de un diccionario con valroes iniciales
        # NOTA: cada estado es un diccionario con los siguientes atributos
        self.estado_inicial = {"solucion":dict(),
                               "DigitosNoAsignados":[i for i in range(0,10)],
                               "col": 0,
                               "acarreo":0
                              }
        
        
    
    def acciones_aplicables(self, estado):
        """
        Calcula las acciones aplicables a partir de un estado
        ARGUMENTOS:

        estado: diccionario que representa un estado del problema con evolucion de la solucion y digitos no asignados 
        
        RETORNA:
        Retorna una lista de parejas (letra, valor) de las asignaciones posibles
        a partir de un estado
        """
        # Se mira en la columna especificada (de derecha a izquierda):
        letrasVisitadas = []
        acc_aplicables = []
        # Verificando en sumandos
        # print("-----[Acciones posibles]-----")
        # print("Columna observada", estado["col"])
        for i in range(len(self.sumandos)):
            # print("SUMANDO %d" %(i+1))
            if existe(self.sumandos[i], estado["col"]):
                # print("Caracter en cuestion: ",self.sumandos[i][-1 - estado["col"]])
                # print("%s aparece en el sumando" %self.sumandos[i][-1 - estado["col"]])
                c = self.sumandos[i][-1 - estado["col"]]
                # Si el caracter no ha sido asignado ni visto anteriormente
                if c not in estado["solucion"].keys() and c not in letrasVisitadas:
                    # print("ACCIONES AGREGADAS")
                    acc_aplicables += [(c,n) for n in estado["DigitosNoAsignados"]]
                    letrasVisitadas.append(c)
            #else:
                # print("No se puede acceder a la columna en el sumando" %self.sumandos[i][-1 - estado["col"]])
        
        # print("\n")
        # Verificando en resultado
        # print("EN RESULTADO")
        if existe(self.resultado, estado["col"]):
            # print("%s aparece en el resultado" %self.resultado[-1 - estado["col"]])
            c = self.resultado[-1 - estado["col"]]
            if c not in estado["solucion"].keys():
                if c not in letrasVisitadas:
                    # print("Se agrega las acciones posibles con %s" %c)
                    acc_aplicables += [(c,n) for n in estado["DigitosNoAsignados"]]
                #else:
                    # print("Letras Visitadas:", letrasVisitadas)
                    # print("%s ya fue visitada")
            #else:
                # print("%s ya esta asignada, solucion = %s" %(c, estado["solucion"]))
        # print("Acciones aplicables:")
        # print(acc_aplicables)
        return acc_aplicables
        
        
    def transicion(self, estado, asignacion):
        """
        Devuelve un diccionario con la nueva asignacion de la forma letra:valor
        
        ARGUMENTOS:
        estado: estado actual de asignaciones de valores a letras
        asignacion: pareja (letra,valor) que se va a agregar en solucion de la forma (letra:valor)
        
        RETORNA:
        un estado que es un diccionario con la pareja adicional y un digito eliminado de la lista de no visitados
        """
        s = deepcopy(estado)
        # Se agrega a la solucion la nueva pareja
        s["solucion"][asignacion[0]] = asignacion[1]
        # Se elimina el digito de la lista de no usados en la asignacion de la letra
        s["DigitosNoAsignados"].remove(asignacion[1])
        # se retorna un nuevo estado, con la solucion actualizada
        MAX_LEN = max([len(self.sumandos[0]), len(self.sumandos[1]), len(self.resultado)])
        # Para que se devuelva a la columna de la izquierda
        s["col"] = (s["col"] + 1) % MAX_LEN # PREGUNTA: SE DEVUELVE UNA VEZ COL LLEGUE AL FINAL?
        #print("Col nuevo estado: ",s["col"])
        col = estado["col"]
        # Calculo de acarreo para siguiente estado
        if estado["col"] < MAX_LEN:
            suma = estado["acarreo"]
            for i in range(2):
                if existe(self.sumandos[i], col):
                    if self.sumandos[i][-1-col] in estado["solucion"].keys():
                        suma += estado["solucion"][self.sumandos[i][-1 - col]]
            s["acarreo"] =  suma // 10

        return s
    
    def test_objetivo(self, estado):
        """
        Determina si el estado ingresado es solucion del problema
        
        ARGUMENTOS:
        estado: estado actual de asignaciones de valores a letras
        
        RETORNA:
        True si estado es un diccionario con una asignacion para todas las letras de la expresion
        tal que la asignacion de numeros es unica y que la suma coincide, False de lo contrario
        """
        # Validar:
        # R1: la suma coincide con resultado (PREGUNTA: considerar acarreo o solo sumar despues de convertir?)
        # R2: Cada letra de los sumandos y de resultado tiene un valor asignado, no hay dos letras con mismo valor
        
        # Verificacion de R2
        letras = set(list(self.sumandos[0])).union(list(self.sumandos[1])).union(list(self.resultado))
        digitsUsed = []
        for L in letras:
            if L not in estado["solucion"].keys():
                #print("NOTA: la letra '%s' no tiene un valor asignado" %L)
                return False
            elif estado["solucion"][L] in digitsUsed:
                #print("NOTA: %d ya fue usado en la solucion" %estado["solucion"][L])
                return False
            else:
                digitsUsed.append(estado["solucion"][L])
        cumpleR2 = True
        
        # Ahora que se confirmó que cada letra tiene un valor, verifiquemos R1
        # Pasando Sumandos a enteros
        sumandosInt = []
        for i in range(2):
            num = ""
            for L in self.sumandos[i]:
                num += str(estado["solucion"][L])
            sumandosInt.append(int(num))
        # Pasando resultado a entero
        num = ""
        for L in self.resultado:
            num += str(estado["solucion"][L])
        intResult = int(num)
        #print("Resultado en numeros: ",intResult)
        cumpleR1 = sum(sumandosInt) == intResult
        
        # Validacion final
        return cumpleR1 and cumpleR2
    
    
    def codigo(self, estado):
        """
        Retorna un identificador unico del estado
        ARGUMENTOS:
            estado: diccionario que representa un estado del problema
        
        RETORNA:
            string con el codigo del estado
        """
        # el codigo se creara como un string de la forma: L1-V1 o L1-V1/L2-V2/.../Ln-Vn
        cod = ""
        inicio = True
        for k in estado["solucion"].keys():
            if inicio:
                cod += k+str("-") + str(estado["solucion"][k])
                inicio = False
            else:
                cod += "/"+k+str("-") + str(estado["solucion"][k])
        return cod
    
    def costo(self, estado, accion):
        return 1


In [85]:
prob = Criptoaritmetica('ab','cd','ef')
prob = Criptoaritmetica('SEND','MORE','MONEY')

Operacion final: AB+CD=EF
Operacion final: SEND+MORE=MONEY


In [ ]:
s = prob.estado_inicial
print("Estado inicial: ",s)

Estado inicial:  {'solucion': {}, 'DigitosNoAsignados': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'col': 0, 'acarreo': 0}


## Pruebas de algoritmos

**Refefinición de la clase nodo**: tiene el atributo costo_camino con el que se calcula el costo de moverse desde la raiz hasta un nodo hijo a partir de una accion

### Depth First Search

In [94]:
%%time
sol = depth_first_search(prob) # Nodo con la solucion

CPU times: user 881 ms, sys: 1e+03 ns, total: 881 ms
Wall time: 879 ms


In [ ]:
s1,s2,r,S1,S2,R=recSolution(prob,dict(find_path(sol)))
toTex(s1,s2,r,S1,S2,R)

<IPython.core.display.Latex object>

### Breadth First Search

In [7]:
%%time
sol2 = breadth_first_search(prob) #este algoritmo tarda demasiado para correr el ejemplo SEND+MORE=MONEY

KeyboardInterrupt: 

In [8]:
s1,s2,r,S1,S2,R=recSolution(prob,dict(find_path(sol2)))
toTex(s1,s2,r,S1,S2,R)

NameError: name 'sol2' is not defined

### Depth Limited Search

In [91]:
%%time
sol3 = depth_limited_search(prob, 10)

CPU times: user 281 ms, sys: 0 ns, total: 281 ms
Wall time: 280 ms


In [ ]:
if type(sol3) == str:
    print(sol3)
else:
    print("Costo de solucion: ",sol3.costo_camino)
    s1,s2,r,S1,S2,R=recSolution(prob,sol3.estado["solucion"])
    toTex(s1,s2,r,S1,S2,R)

Costo de solucion:  8


<IPython.core.display.Latex object>

## Iterative Deepeneing Search

In [ ]:
%%time
max_depth = 10
sol4 = iterative_deepening_search(prob, max_depth)

Con profundidad 0 ...
Terminado!
-----------
Con profundidad 1 ...
Terminado!
-----------
Con profundidad 2 ...
Terminado!
-----------
Con profundidad 3 ...
Terminado!
-----------
Con profundidad 4 ...
Terminado!
-----------
Con profundidad 5 ...
Terminado!
-----------
Con profundidad 6 ...
Terminado!
-----------
Con profundidad 7 ...
Terminado!
-----------
Con profundidad 8 ...
CPU times: user 10min 45s, sys: 13.4 ms, total: 10min 45s
Wall time: 10min 45s


In [12]:
print("Con iterative deepening search, (max_depth = ",max_depth,")")
if sol4 is not None:
    if sol4 == "cutoff":
        print("Cutoff alcanzado")
    else:
        print("La solucion encontrada es:")
        s1,s2,r,S1,S2,R=recSolution(prob,sol4.estado["solucion"])
        toTex(s1,s2,r,S1,S2,R)
else:
    print("No se encontro una solucion")

Con iterative deepening search, (max_depth =  10 )
La solucion encontrada es:


<IPython.core.display.Latex object>

### Lista Prioritaria y refefinicion de costo
Se trajo la clase **ListaPrioritaria** la cual nos permite buscar el elemento más prioritario (el de menor costo) de la lista de acciones posibles. Ahora definamos una función de costo conveniente para el problema:

In [15]:
def getFrequencyCost(sumandos, resultado, sol):
    # Calcula un costo dependiendo de la frecuencia de cada una de las letras que faltan por reemplazar
    # Entre más frecuente menor es el costo (es decir más prioritario segun ListaPrioritaria)
    # ARGUMENTOS:
    #   sumandos: lista con los dos strings que son los sumandos
    #   resultado: string del resultado de sumar los dos sumandos
    # sol: diccionario con asignaciones de valores a letras
    # RETORNA:
    #   diccionario con costos por seleccionar una letra a partir de su frecuencia
    totalLetters = len(sumandos[0]) + len(sumandos[1]) + len(resultado)
    freqCost = dict()
    for i in range(2):
        for c in sumandos[i]:
            if c not in sol.keys():
                if c not in freqCost.keys():
                    freqCost[c] = totalLetters
                else:
                    freqCost[c] -= 1
    for c in resultado:
        if c not in freqCost.keys() and c not in sol.keys():
            freqCost[c] = totalLetters
        elif c in freqCost.keys() and c not in sol.keys():
            freqCost[c] -= 1

    return freqCost

    

def costoAsignacion(self, estado, accion):
    """
    Define el costo de asignar un valor a una letra.
    Como la clase ListaPrioritaria devuelve el objeto más prioritario con el menor costo entonces
    la mejor opcion debe tener el mejor costo

    ARGUMENTOS:
        self: objeto que representa al problema
        estado: estado desde el cual se observa la accion posible
        accion: pareja (letra, valor) que representa la accion realizada desde estado

    RETORNA:
        float igual al costo de hacer la respectiva accion.
    """
    # Primero se va a buscar la frecuencia de cada letra. Vamos a asumir que entre
    # más letras se reemplacen menor operaciones se tendrá que hacer.
    n1 = len(self.sumandos[0])
    n2 = len(self.sumandos[1])
    cF = getFrequencyCost(self.sumandos, self.resultado, estado["solucion"])

    # Segundo, filtramos todas las acciones cuyo costo es el minimo
    costo = cF[accion[0]]
    #print("Solucion inicial: ",estado["solucion"])
    sol2 = deepcopy(estado["solucion"]) # posible solucion
    sol2[accion[0]] = accion[1]
    #print("Sol2: ",sol2)
    # Buscamos desde la derecha hacia la izquierda
    for j in range(2):
        for i in range(len(self.sumandos[j])):
            # Despeje para el primer sumando
            if j == 0:
                c = self.sumandos[0][-1-i]
                # Si se encontro la letra de la accion
                if c == accion[0]:
                    # Se verifica si en la columna se puede despejar
                    if existe(self.sumandos[1], i) and existe(self.resultado,i):
                        costo = costo/2.0
                        # Se verifica la suma y el resultado son iguales con la asignacion
                        if match(self, sol2, -1-i):
                            costo = costo/2.0 # maxima prioridad
                    # Despues de reducir el costo (si se da) de cualquier forma retornamos su valor
                    return costo
            if j == 1:
                c = self.sumandos[1][-1-i]
                # Si se encontro la letra de la accion
                if c == accion[0]:
                    # Se verifica si en la columna se puede despejar
                    if existe(self.sumandos[0], i) and existe(self.resultado,i):
                        costo = costo/2.0
                        # Se verifica la suma y el resultado son iguales con la asignacion
                        if match(self, sol2, -1-i):
                            costo = costo/2.0 # maxima prioridad
                    # Despues de reducir el costo (si se da) de cualquier forma retornamos su valor
                    return costo
    # Ultima verificacion por si la letra no estaba en sumandos pero sí en resultado
    for i in range(len(self.resultado)):
        c = self.resultado[-1-i]
        if c == accion[0]:
            if existe(self.sumandos[0], i) and existe(self.sumandos[1], i):
                costo = costo/2.0
            if match(self, sol2, -1-i):
                costo = costo/2.0
            return costo

def existe(s,idx):
    # s: string del que se quiere validar si existe el caracter
    # idx: indice de iterador
    return idx < len(s)

def match(self,solucion, col):
    # verifica si la suma calculada con los sumandos coincide con el valor del resultado en ultima fila (sin acarreo)
    # solucion: diccionario con parejas (letra, valor) que ya fueron asignados
    # col: entero igual al indice de la columna en la que se calcula la suma y se compara con resultado de esa columna
    if self.sumandos[0][col] in solucion.keys() and self.sumandos[1][col] in solucion.keys() and self.resultado[col] in solucion.keys():
        suma = (solucion[self.sumandos[0][col]] + solucion[self.sumandos[1][col]]) %10
        if suma == solucion[self.resultado[col]]:
            return True
        else:
            return False
    return False


In [16]:
setattr(Criptoaritmetica,"costo", costoAsignacion)

## Best first search

In [17]:
def best_first_search(problema,f):
    # implementacion de depth first search con funcion de costo

    # ARGUMENTOS:
    # problema: problema el cual se trabaja
    # f: funcion de costo
    
    #Inicializacion
    if f is not None:
        setattr(problema, "costo", f)
        
    s = problema.estado_inicial
    cod = problema.codigo(s)
    
    nodo = Nodo(s, None, None,0, cod) # nodo raiz
    frontera = ListaPrioritaria() # lista prioritaria
    frontera.push(nodo, 0)
    explorados = {}
    explorados[cod] = 0
    
    # Iteraciones de busqueda
    while not frontera.is_empty():
        nodo = frontera.pop()
        
        if problema.test_objetivo(nodo.estado):
            return nodo
        
        for hijo in expand(problema, nodo):
            # Se busca cada uno de los hijos del nodo actual con estado, codigo y costo hasta hijo
            s = hijo.estado # estado de hijo
            cod = problema.codigo(s) #codigo del estado del hijo
            c = hijo.costo_camino # costo del camino hasta hijo
            
            # Si el hijo no ha sido explorado o el costo hasta hijo es menor (camino optimo)
            if cod not in explorados.keys() or c < explorados[cod]:
                frontera.push(hijo, c)
                explorados[cod] = c
    return None


In [18]:
%%time
sol5 = best_first_search(prob, costoAsignacion)

CPU times: user 1min, sys: 872 ms, total: 1min 1s
Wall time: 1min 1s


In [19]:
if sol5 is not None:
    if sol5 == "cutoff":
        print("Cutoff alcanzado")
    else:
        print("La solucion encontrada es:")
        s1,s2,r,S1,S2,R=recSolution(prob,sol5.estado["solucion"])
        toTex(s1,s2,r,S1,S2,R)
else:
    print("No se encontro una solucion")

La solucion encontrada es:


<IPython.core.display.Latex object>